# Example 4.7: CRR Model with parameter uncertainty

First, include the necessary functions.
The linear programming solver requires the gurobi package!

In [1]:
source(file="functions.r")

Warning message:
"package 'linprog' was built under R version 3.6.2"Loading required package: lpSolve
Warning message:
"package 'lpSolve' was built under R version 3.6.2"Warning message:
"package 'gurobi' was built under R version 3.6.2"Loading required package: slam
Warning message:
"package 'slam' was built under R version 3.6.2"

First, we fix the model parameters.
Nr. of time steps $n$, different probabilities for the computations and $S_{t_0}$.

In [2]:
n<-10 # Nr.of time steps
N<-100 # Nr. of different P's
S_0<-100

Next, we choose the up-factor $u$, the down-factor $d$ and the interval of possible probabilities $[0.4,0.6]$

In [3]:
u<-rep(1.1,N)
d<-rep(1/u,N)
p<-seq(0.4,0.6,length=N)

We assign the possible values that are fixed in the model
and corresponding admissible probabilities

In [4]:
x<-matrix(0,N,n/2+1)
y<-matrix(0,N,n+1)
# probabilities
prob_set<-array(0,dim=c(N,(n/2)+1,n+1))
for(i in 1:N){
  # Assigning Values
  for(l in 0:(n/2)){
    x[i,l+1]<-S_0*u[i]^l*d[i]^(n/2-l)
  }
  for(l in 0:(n)){
    y[i,l+1]<-S_0*(u[i]^l)*d[i]^(n-l)
  }
  for(j in 1:((n/2)+1)){
    for(k in 0:(n/2)){
      # Assigning Probabilities
      prob_set[i,j,k+j]<-dbinom(j-1,n/2,p[i])*dbinom(k,n/2,p[i])
    }
  }
}

We check if statistical arbitrage is available in the market

In [5]:
check_statarb(x[10,],y[10,],prob_set[10,,],S_0)

[1] "No Statistical Arbitrage!"


We define our payoff function and compute associated bound

In [6]:
# Function Definition
func1<-function(x,y){max(0.5*(x+y)-S_0,0)}

In [7]:
# Robust Bounds
limit <- Inf
lower_bound_nosa<-super_rep(x,y,prob_set,func=func1,stat_arb=F,lower=T,S_0,limit)
lower_bound<-super_rep_robust(x,y,prob_set,func=func1,stat_arb=T,lower=T,S_0,limit)
upper_bound<-super_rep_robust(x,y,prob_set,func=func1,stat_arb=T,lower=F,S_0,limit)
upper_bound_nosa<-super_rep(x,y,prob_set,func=func1,stat_arb=F,lower=F,S_0,limit)

In [8]:
# Computation of the Bounds
limit<-Inf
# Robust Bounds
lower_bound_nosa<-super_rep(x,y,prob_set,func=func1,stat_arb=F,lower=T,S_0,limit)
lower_bound<-super_rep_robust(x,y,prob_set,func=func1,stat_arb=T,lower=T,S_0,limit)
upper_bound<-super_rep_robust(x,y,prob_set,func=func1,stat_arb=T,lower=F,S_0,limit)
upper_bound_nosa<-super_rep(x,y,prob_set,func=func1,stat_arb=F,lower=F,S_0,limit)

The price interval that excludes robust statistical arbitrage

In [9]:
print(lower_bound$d)
print(upper_bound$d)

[1] 9.354121
[1] 9.886701


The price interval that excludes robust arbitrage

In [10]:
print(lower_bound_nosa$d)
print(upper_bound_nosa$d)

[1] 4.761905
[1] 27.41413


Next, we perform the simulations

First we assume the traded price of $c$ is 5, then we use the strategy attaining the lower bound to make a profit 

In [11]:
Delta_0_lower<-lower_bound$Delta_0
Delta_1_lower<-splinefun(lower_bound$x,lower_bound$Delta_1,method="natural")

## Statistical Arbitrage
nr_gains<-0
nr_losses<-0
d_lower<-lower_bound$d
gain<-0
Nr_sim<-1000000
best<-0
worst<-0
for(i in 1:Nr_sim){
  random_p<-rbinom(n,1,runif(1,0.4,0.6))
  S<-S_0*cumprod(u[1]*(random_p==1)+d[1]*(random_p==0))
  f<-(func1(S[n/2],S[n])-Delta_0_lower*(S[n/2]-S_0)-Delta_1_lower(S[n/2])*(S[n]-S[n/2])-5)
  gain<-gain+f
  if(f<0){nr_losses<-nr_losses+1}
  if(f>0){nr_gains<-nr_gains+1}
  if(f>best){best<-f}
  if(f<worst){worst<-f}
}
print("Average Profit")
gain/Nr_sim
print("Best Gain")
best
print("Worst Loss")
worst
print("% Loss Scenarios")
nr_losses/Nr_sim
print("% Gain Scenarios")
nr_gains/Nr_sim

[1] "Average Profit"


[1] 4.694019

[1] "Best Gain"


[1] 88.15069

[1] "Worst Loss"


[1] -45.97068

[1] "% Loss Scenarios"


[1] 0.556696

[1] "% Gain Scenarios"


[1] 0.443304

Next, we assume the price of $c$ is $ 9$

In [12]:
## Statistical Arbitrage
nr_gains<-0
nr_losses<-0
d_lower<-lower_bound$d
gain<-0
Nr_sim<-1000000
best<-0
worst<-0
for(i in 1:Nr_sim){
  random_p<-rbinom(n,1,runif(1,0.4,0.6))
  S<-S_0*cumprod(u[1]*(random_p==1)+d[1]*(random_p==0))
  f<-(func1(S[n/2],S[n])-Delta_0_lower*(S[n/2]-S_0)-Delta_1_lower(S[n/2])*(S[n]-S[n/2])-9)
  gain<-gain+f
  if(f<0){nr_losses<-nr_losses+1}
  if(f>0){nr_gains<-nr_gains+1}
  if(f>best){best<-f}
  if(f<worst){worst<-f}
}
print("Average Profit")
gain/Nr_sim
print("Best Gain")
best
print("Worst Loss")
worst
print("% Loss Scenarios")
nr_losses/Nr_sim
print("% Gain Scenarios")
nr_gains/Nr_sim

[1] "Average Profit"


[1] 0.6506975

[1] "Best Gain"


[1] 84.15069

[1] "Worst Loss"


[1] -49.97068

[1] "% Loss Scenarios"


[1] 0.615284

[1] "% Gain Scenarios"


[1] 0.384716

Next, we assume the price of $c$ is $10$, then we use the strategy attaining the upper bound

In [13]:
Delta_0_upper<-upper_bound$Delta_0
Delta_1_upper<-splinefun(upper_bound$x,upper_bound$Delta_1,method="natural")


nr_gains<-0
nr_losses<-0
d_upper<-upper_bound$d
gain<-0
Nr_sim<-1000000
best<-0
worst<-0
for(i in 1:Nr_sim){
  random_p<-rbinom(n,1,runif(1,0.4,0.6))
  S<-S_0*cumprod(u[1]*(random_p==1)+d[1]*(random_p==0))
  f<-(-func1(S[n/2],S[n])+Delta_0_upper*(S[n/2]-S_0)+Delta_1_upper(S[n/2])*(S[n]-S[n/2])+10)
  gain<-gain+f
  if(f<0){nr_losses<-nr_losses+1}
  if(f>0){nr_gains<-nr_gains+1}
  if(f>best){best<-f}
  if(f<worst){worst<-f}
}
print("Average Profit")
gain/Nr_sim
print("Best Gain")
best
print("Worst Loss")
worst
print("% Loss Scenarios")
nr_losses/Nr_sim
print("% Gain Scenarios")
nr_gains/Nr_sim

[1] "Average Profit"


[1] 0.392579

[1] "Best Gain"


[1] 52.53063

[1] "Worst Loss"


[1] -98.92762

[1] "% Loss Scenarios"


[1] 0.382437

[1] "% Gain Scenarios"


[1] 0.617563

Next, we assume the price of $c$ is $20$

In [14]:
nr_gains<-0
nr_losses<-0
d_upper<-upper_bound$d
gain<-0
Nr_sim<-1000000
best<-0
worst<-0
for(i in 1:Nr_sim){
  random_p<-rbinom(n,1,runif(1,0.4,0.6))
  S<-S_0*cumprod(u[1]*(random_p==1)+d[1]*(random_p==0))
  f<-(-func1(S[n/2],S[n])+Delta_0_upper*(S[n/2]-S_0)+Delta_1_upper(S[n/2])*(S[n]-S[n/2])+20)
  gain<-gain+f
  if(f<0){nr_losses<-nr_losses+1}
  if(f>0){nr_gains<-nr_gains+1}
  if(f>best){best<-f}
  if(f<worst){worst<-f}
}
print("Average Profit")
gain/Nr_sim
print("Best Gain")
best
print("Worst Loss")
worst
print("% Loss Scenarios")
nr_losses/Nr_sim
print("% Gain Scenarios")
nr_gains/Nr_sim

[1] "Average Profit"


[1] 10.37817

[1] "Best Gain"


[1] 62.53063

[1] "Worst Loss"


[1] -88.92762

[1] "% Loss Scenarios"


[1] 0.209598

[1] "% Gain Scenarios"


[1] 0.790402